## Heavy Lifter Collecting the Non-Hex Layers
Needs to be cleaned up as it contains a good deal of cannibalized code and input from Claude Code.

In [ ]:
# Collection of functions needing to be initialized in order to be reused below. 

import requests
import json
import pandas as pd
from pathlib import Path
import zipfile
import io

# Basic set up for grabbind data from ESRIs REST Server
# Filtering using "where" is optional, defaults to getting everything with the 1=1
# Need to update with bounding box/envelop input option
def scrape_esri_rest(service_url: str, where: str = "1=1", out_fields: str = "*",
                     batch_size: int = 1000, out_sr: int = 4326) -> pd.DataFrame:
    """
    Scrape all features from an Esri REST service endpoint.

    Parameters
    ----------
    service_url : str
        Full URL to the feature layer, e.g.
        "https://services.arcgis.com/.../FeatureServer/0"
    where : str
        SQL where clause to filter features (default "1=1" = all).
    out_fields : str
        Comma-separated field names or "*" for all.
    batch_size : int
        Records per request (server may cap this, typically 1000-2000).
    out_sr : int
        Output spatial reference WKID (default 4326 = WGS84).

    Returns
    -------
    pd.DataFrame with all attributes and geometry as a column.
    """
    query_url = f"{service_url.rstrip('/')}/query"

    # First, get the total count
    count_params = {
        "where": where,
        "returnCountOnly": True,
        "f": "json",
    }
    resp = requests.get(query_url, params=count_params)
    resp.raise_for_status()
    total = resp.json().get("count", 0)
    print(f"Total features to fetch: {total}")

    # Paginate through all features
    all_features = []
    offset = 0

    while offset < total:
        params = {
            "where": where,
            "outFields": out_fields,
            "outSR": out_sr,
            "f": "json",
            "resultOffset": offset,
            "resultRecordCount": batch_size,
            "returnGeometry": True,
        }
        resp = requests.get(query_url, params=params)
        resp.raise_for_status()
        data = resp.json()

        features = data.get("features", [])
        if not features:
            break

        for feat in features:
            row = feat.get("attributes", {})
            geom = feat.get("geometry")
            if geom:
                row["geometry"] = json.dumps(geom)
            all_features.append(row)

        offset += len(features)
        print(f"  Fetched {offset}/{total}")

    df = pd.DataFrame(all_features)
    print(f"Done — {len(df)} records retrieved.")
    return df


# Geometry helpers
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, LinearRing
from shapely.validation import make_valid


def parse_geometry(g):
    """
    Parse an Esri ring-based polygon geometry string into a Shapely geometry.

    Handles both single-polygon and multi-part (MultiPolygon) cases by
    inspecting ring winding order:
      - Esri outer rings  → clockwise      (LinearRing.is_ccw == False)
      - Esri hole rings   → counter-clockwise (LinearRing.is_ccw == True)

    If there are multiple outer rings the result is a MultiPolygon.
    """
    try:
        parsed = json.loads(g)
        rings = parsed.get("rings")
        if not rings:
            return None

        outers, holes = [], []
        for ring in rings:
            lr = LinearRing(ring)
            (holes if lr.is_ccw else outers).append(ring)

        # Fallback: if winding detection gives no outers, treat first ring as outer
        if not outers:
            outers, holes = [rings[0]], rings[1:]

        if len(outers) == 1:
            geom = Polygon(outers[0], holes)
        else:
            # Multiple outer rings → MultiPolygon; assign holes to their parent
            polys = []
            for outer in outers:
                outer_poly = Polygon(outer)
                assigned = [h for h in holes if Polygon(h).within(outer_poly)]
                polys.append(Polygon(outer, assigned))
            geom = MultiPolygon(polys)

        return make_valid(geom)

    except Exception as e:
        print(f"  parse_geometry error: {e}")
        return None


# Save the json
def save_esri(s, out_dir, layer_name, format="gpkg"):
    # Convert Esri geometry to shapely and save as GeoPackage
    df_temp = s[s["geometry"].notna()].copy()
    df_temp["_geom"] = df_temp["geometry"].apply(parse_geometry)
    df_temp = df_temp[df_temp["_geom"].notna()]

    gdf = gpd.GeoDataFrame(
        df_temp.drop(columns=["geometry", "_geom"]),
        geometry=df_temp["_geom"].tolist(),
        crs="EPSG:4326",
    )

    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    drivers = {"gpkg": "GPKG", "geojson": "GeoJSON"}
    gdf.to_file(out_dir / f"{layer_name}.{format}", driver=drivers[format])
    print(f"Saved to {out_dir.resolve()} ({len(gdf)} features)")

# For use with direct download links that serve up zipped folders
def download_zip(url: str, out_dir) -> Path:
    """
    Download a zip archive from a URL and extract it to out_dir.

    Parameters
    ----------
    url     : str   Direct download URL for the .zip file.
    out_dir : str | Path  Destination directory (created if needed).

    Returns
    -------
    Path  The resolved output directory.
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    filename = url.split("/")[-1]
    print(f"Downloading {filename}...")
    r = requests.get(url, stream=True)
    r.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(r.content)) as zf:
        zf.extractall(out_dir)
        extracted = zf.namelist()

    print(f"Extracted {len(extracted)} files → {out_dir.resolve()}")
    return out_dir

In [ ]:
# Prepping and running Tippecanoe with local Docker container

import subprocess
from pathlib import Path

def to_docker_path(p):
    return str(Path(p).resolve()).replace("\\", "/")

base  = Path("..").resolve()
tiles = base / "tiles"
tiles.mkdir(exist_ok=True)

# Build local tippecanoe image if not already built
image_check = subprocess.run(["docker", "image", "inspect", "tippecanoe"], capture_output=True)
if image_check.returncode != 0:
    print("Building tippecanoe Docker image (one-time, takes ~2 min)...")
    build = subprocess.run(
        ["docker", "build", "-t", "tippecanoe",
         "-f", f"{to_docker_path(base)}/Dockerfile.tippecanoe", str(base)],
        capture_output=True, text=True
    )
    print("Image built successfully" if build.returncode == 0 else f"Build failed:\n{build.stderr}")
else:
    print("tippecanoe image already exists")


def run_tippecanoe(layers, base=base, tiles=tiles):
    """
    Tile a list of GeoJSON layers using Tippecanoe via Docker.

    Parameters
    ----------
    layers : list of (layer_name, geojson_path_relative_to_base, min_zoom, max_zoom)
    base   : Path  root directory mounted into Docker as /data
    tiles  : Path  output directory for .mbtiles files
    """
    for layer_name, geojson_rel, zmin, zmax in layers:
        cmd = [
            "docker", "run", "--rm",
            "-v", f"{to_docker_path(base)}:/data",
            "tippecanoe",
            "tippecanoe",
            "-o", f"/data/tiles/{layer_name}.mbtiles",
            f"-Z{zmin}", f"-z{zmax}",
            "--layer", layer_name,
            "--drop-densest-as-needed",
            "--maximum-tile-bytes", "500000",
            "--force",
            f"/data/{geojson_rel}"
        ]
        print(f"Running Tippecanoe: {layer_name}...")
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            size = (tiles / f"{layer_name}.mbtiles").stat().st_size / 1e6
            print(f"  Done — {size:.1f} MB")
        else:
            print(f"  Error:\n{result.stderr}")

tippecanoe image already exists


In [ ]:
# Functions to ship off the tiles to Mapbox


import boto3
import requests
import os
import time
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

MAPBOX_USERNAME = "keystonecarto"
st_upload_write = os.environ["MAPBOX_SECRET_WRITE"]
st_upload_read  = os.environ["MAPBOX_SECRET_READ"]

def upload_to_mapbox(mbtiles_path: Path, tileset_id: str, st_w, st_r):
    """
    Upload an .mbtiles file to Mapbox as a tileset.

    Parameters
    ----------
    mbtiles_path : Path   Local path to the .mbtiles file.
    tileset_id   : str   Tileset name without username prefix, e.g. 'ny_dec_lands'.
                         Final tileset will be '{username}.{tileset_id}'.
    """
    tileset_full = f"{MAPBOX_USERNAME}.{tileset_id}"
    print(f"Uploading {mbtiles_path.name} → {tileset_full}")

    # Step 1 — get temporary S3 staging credentials
    creds_resp = requests.get(
        f"https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}/credentials",
        params={"access_token": st_w}
    )
    creds_resp.raise_for_status()
    creds = creds_resp.json()

    # Step 2 — upload file to Mapbox's S3 staging bucket
    print("  Uploading to S3 staging...")
    s3 = boto3.client(
        "s3",
        aws_access_key_id=creds["accessKeyId"],
        aws_secret_access_key=creds["secretAccessKey"],
        aws_session_token=creds["sessionToken"],
        region_name="us-east-1",
    )
    s3.upload_file(str(mbtiles_path), creds["bucket"], creds["key"])
    print("  S3 upload complete")

    # Step 3 — trigger Mapbox processing
    upload_resp = requests.post(
        f"https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}",
        params={"access_token": st_w},
        json={
            "url": f"https://{creds['bucket']}.s3.amazonaws.com/{creds['key']}",
            "tileset": tileset_full,
        }
    )
    upload_resp.raise_for_status()
    upload_id = upload_resp.json()["id"]
    print(f"  Processing started (upload id: {upload_id})")

    # Step 4 — poll until complete or failed
    # Modify this with a time out, high risk of infinite loop and really it's unnecessary
    while True:
        status_resp = requests.get(
            f"https://api.mapbox.com/uploads/v1/{MAPBOX_USERNAME}/{upload_id}",
            params={"access_token": st_r}
        )
        status = status_resp.json()
        if status.get("complete"):
            print(f"  Done — tileset ready: mapbox://{tileset_full}")
            return tileset_full
        if status.get("error"):
            raise RuntimeError(f"Mapbox upload failed: {status['error']}")
        print(f"  Processing... ({status.get('progress', 0)*100:.0f}%)")
        time.sleep(10)

## Start of the Data Collection

In [8]:
# Hunting Units

# New Jersey

service_url = "https://mapsdep.nj.gov/arcgis/rest/services/Features/Environmental_admin/MapServer/17"
out_dir = Path("../data/hunting_units/nj/")
layer_name = 'deer_management_zones'
file_type = 'gpkg'

df = scrape_esri_rest(service_url, batch_size=2000)

# Convert geojson to geopackage and save
save_esri(df, out_dir, layer_name, file_type)


# New York

service_url = "https://services6.arcgis.com/DZHaqZm9cxOD4CWM/ArcGIS/rest/services/Wildlife_Management_Units/FeatureServer/0"
out_dir = Path("../data/hunting_units/ny/")
layer_name = 'wildlife_management_units'
file_type = 'gpkg'

df = scrape_esri_rest(service_url, batch_size=2000)

# Convert geojson to geopackage and save
save_esri(df, out_dir, layer_name, file_type)


# Pennsylvania
source_url = "https://www.pasda.psu.edu/download/pgc/PGC_BNDWildlifeManagementUnits2021.zip"
dest_folder = "../data/hunting_units/pa/"

download_zip(
    source_url,
    dest_folder
)

Total features to fetch: 90
  Fetched 90/90
Done — 90 records retrieved.
Saved to C:\data\repos\onx_hunt\data\hunting_units\nj (90 features)
Total features to fetch: 92
  Fetched 92/92
Done — 92 records retrieved.
Saved to C:\data\repos\onx_hunt\data\hunting_units\ny (92 features)
Extracted 9 files → C:\data\repos\onx_hunt\data\hunting_units\pa


WindowsPath('../data/hunting_units/pa')

In [ ]:
# Public Lands

# Scrape NJ
# Going to go easy on their server and filter extra out since this is based off of parcels

service_url = "https://mapsdep.nj.gov/arcgis/rest/services/Features/Land/MapServer/65"
out_dir = Path("../data/public_lands/nj/")
layer_name = 'parks_and_forests'

filt_col = 'USE_LABEL'
filt_vals = ['State Park', 'State Forest']
where = f'"{filt_col}" IN ({", ".join(f"\'{v}\'" for v in filt_vals)})'

print(where)

df = scrape_esri_rest(service_url, batch_size=2000, where=where)


# Convert geojson to geopackage and save
save_esri(df, out_dir, layer_name)

# Scrape NYS DEC Lands (state forests, wildlife mgmt areas, etc.)
service_url = "https://services6.arcgis.com/DZHaqZm9cxOD4CWM/ArcGIS/rest/services/NYS_DEC_Lands/FeatureServer/0"
out_dir = Path("../data/public_lands/ny/")
layer_name = 'dec_lands'
file_type = 'gpkg'

df = scrape_esri_rest(service_url, batch_size=2000)

# Convert geojson to geopackage and save
save_esri(df, out_dir, layer_name, file_type)


# Pennsylvania

source_url = "https://www.pasda.psu.edu/download/pgc/PGC_StateGameland202301.zip"
dest_folder = "../data/public_lands/pa/"

download_zip(
    source_url,
    dest_folder
)


"USE_LABEL" IN ('State Park', 'State Forest')
Total features to fetch: 8726
  Fetched 2000/8726
  Fetched 4000/8726
  Fetched 6000/8726
  Fetched 8000/8726
  Fetched 8726/8726
Done — 8726 records retrieved.
Saved to C:\data\repos\onx_hunt\data\public_lands\nj (8726 features)
Total features to fetch: 3233
  Fetched 2000/3233
  Fetched 3233/3233
Done — 3233 records retrieved.
Saved to C:\data\repos\onx_hunt\data\public_lands\ny (3233 features)
Extracted 9 files → C:\data\repos\onx_hunt\data\public_lands\pa


WindowsPath('../data/public_lands/pa')

In [ ]:
# Harvest Data
# Just PA for now, other states are all in PDFs

service_url = 'https://services1.arcgis.com/k8yxvICm95iIFicb/ArcGIS/rest/services/Historic_Harvest_Data_for_Dashboards/FeatureServer/0'
out_dir = Path("../data/misc/harvest")
layer_name = 'harvest_v1'

df = scrape_esri_rest(service_url, batch_size=2000)

# Convert geojson to geopackage and save
save_esri(df, out_dir, layer_name)

Total features to fetch: 378
  Fetched 378/378
Done — 378 records retrieved.
Saved to C:\data\repos\onx_hunt\data\misc\harvest (362 features)


In [11]:
# Census Data

import geopandas as gpd
from pathlib import Path

state_fips = ['36', '34', '42']  # NY, NJ, PA
out_dir = Path("../data/census")
out_dir.mkdir(parents=True, exist_ok=True)

# States — Census Cartographic Boundary (500k = simplified, good for display)
states = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_500k.zip")
states = states[states['STATEFP'].isin(state_fips)]
states.to_file(out_dir / "states.geojson", driver="GeoJSON")
print(f"States: {len(states)} features saved")

# Counties
counties = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_county_500k.zip")
counties = counties[counties['STATEFP'].isin(state_fips)]
counties.to_file(out_dir / "counties.geojson", driver="GeoJSON")
print(f"Counties: {len(counties)} features saved")


# Create the Mask to highlight the region

import geopandas as gpd
from shapely.geometry import box
from shapely.ops import unary_union
from pathlib import Path

states = gpd.read_file("../data/census/states.geojson")

# Large box — covers visible map area well beyond the 3 states
world_box = box(-180, -85, 180, 85)

# Union the 3 states into one shape, then punch them out of the box
states_union = unary_union(states.geometry)
mask_geom = world_box.difference(states_union)

mask_gdf = gpd.GeoDataFrame({'geometry': [mask_geom]}, crs='EPSG:4326')
out_dir = Path("../data/census")
mask_gdf.to_file(out_dir / "state_mask.geojson", driver="GeoJSON")
print(f"Mask saved — {(out_dir / 'state_mask.geojson').stat().st_size / 1e3:.1f} KB")

run_tippecanoe([
    ("state_mask", "data/census/state_mask.geojson", 2, 7),
])

upload_to_mapbox(tiles / "state_mask.mbtiles", "state_mask", st_upload_write, st_upload_read)

States: 3 features saved
Counties: 150 features saved
Mask saved — 169.5 KB
Running Tippecanoe: state_mask...
  Done — 1.1 MB
Uploading state_mask.mbtiles → keystonecarto.state_mask
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls48e540jx11mnqz4qxgv63)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.state_mask


'keystonecarto.state_mask'

In [ ]:
# Generating the tiles

run_tippecanoe([
    ("census_states",   "data/census/states.geojson",   2, 10),
    ("census_counties", "data/census/counties.geojson",  4, 10),
])


for layer in ["census_states", "census_counties"]:
    upload_to_mapbox(tiles / f"{layer}.mbtiles", layer, st_upload_write, st_upload_read)

Running Tippecanoe: census_states...
  Done — 0.2 MB
Running Tippecanoe: census_counties...
  Done — 0.8 MB
Uploading census_states.mbtiles → keystonecarto.census_states
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4af6l1ksq1mqztd2nwjhk)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.census_states
Uploading census_counties.mbtiles → keystonecarto.census_counties
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4ay1308n41nlda9kfqsdn)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.census_counties


In [13]:
# Seasonal Roads: State forest roads that are normally closed to vehicle traffic, 
# which will be opened to hunters between the dates for the current year's hunting season.

service_url = "https://maps.dcnr.pa.gov/agsprod/rest/services/BOF/HuntStateForest/MapServer/9"
out_dir = Path("../data/misc/")
layer_name = 'seasonal_rds'
file_type = 'gpkg'

df = scrape_esri_rest(service_url, batch_size=2000)

# Convert geojson to geopackage and save
save_esri(df, out_dir, layer_name, file_type)

Total features to fetch: 288
  Fetched 288/288
Done — 288 records retrieved.
Saved to C:\data\repos\onx_hunt\data\misc (0 features)


In [14]:
# Chronic Wasting Disease


# # Could make a time series animation or slider out of the images

# # National
# source_url = 'https://www.sciencebase.gov/catalog/file/get/58068050e4b0824b2d1d415d?f=__disk__f8/ad/16/f8ad16fb7ebd525ff13433fd73d569e5da41d8b6'
# dest_folder = "../data/misc/national/"

# download_zip(
#     source_url,
#     dest_folder
# )

service_url = "https://pgcmaps.pa.gov/arcgis/rest/services/PGC/DMAs/MapServer/0"
out_dir = Path("../data/misc/")
layer_name = 'cwd_disease_management_area'
file_type = 'geojson'

df = scrape_esri_rest(service_url, batch_size=2000)

# Convert geojson to geopackage and save
save_esri(df, out_dir, layer_name, file_type)

Total features to fetch: 9
  Fetched 9/9
Done — 9 records retrieved.
Saved to C:\data\repos\onx_hunt\data\misc (9 features)


In [ ]:
# Get points of interest from OSM using the API

import requests, json
from pathlib import Path

# Bounding box: NY/NJ/PA  (S, W, N, E)
# Remove this hardcoded value and replace
bbox = "38.9,-80.5,45.1,-71.8"

query = f"""
[out:json][timeout:60];
(
  node["shop"="sports"]({bbox});
  node["shop"="firearm"]({bbox});
  node["leisure"="archery"]({bbox});
  node["sport"="archery"]({bbox});
);
out body;
"""

r = requests.post("https://overpass-api.de/api/interpreter", data={"data": query})
elements = r.json()["elements"]

features = []
for el in elements:
    tags = el.get("tags", {})
    features.append({
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [el["lon"], el["lat"]]},
        "properties": {
            "name":    tags.get("name", ""),
            "shop":    tags.get("shop", ""),
            "sport":   tags.get("sport", ""),
            "leisure": tags.get("leisure", ""),
            "phone":   tags.get("phone", ""),
            "website": tags.get("website", ""),
            "addr":    tags.get("addr:street", ""),
        }
    })

out_dir = Path("../data/poi")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "hunting_stores.geojson"
with open(out_path, "w") as f:
    json.dump({"type": "FeatureCollection", "features": features}, f)

print(f"{len(features)} POIs saved to {out_path.resolve()}")

# Clean up the excess that fall outside the detailed border
import geopandas as gpd
from pathlib import Path

poi = gpd.read_file("../data/poi/hunting_stores.geojson")
states = gpd.read_file("../data/census/states.geojson")

# Spatial filter — keep only points that fall within the 3 states
poi_clipped = gpd.sjoin(poi, states[['geometry']], how='inner', predicate='within')
poi_clipped = poi_clipped.drop(columns=['index_right'])

out_path = Path("../data/poi/hunting_stores.geojson")
poi_clipped.to_file(out_path, driver="GeoJSON")
print(f"{len(poi)} → {len(poi_clipped)} POIs after clipping to NY/NJ/PA")



# Alternate Tippecanoe method
# POI — no feature dropping, keep all points at all zooms
# No real need for clustering in this iteration
layer_name = "hunting_stores"
geojson_rel = "data/poi/hunting_stores.geojson"

cmd = [
    "docker", "run", "--rm",
    "-v", f"{to_docker_path(base)}:/data",
    "tippecanoe", "tippecanoe",
    "-o", f"/data/tiles/{layer_name}.mbtiles",
    "-Z2", "-z14",
    "--layer", layer_name,
    "-r1",       # rate=1: keep every feature, no dropping
    "--force",
    f"/data/{geojson_rel}"
]
import subprocess
result = subprocess.run(cmd, capture_output=True, text=True)
if result.returncode == 0:
    size = (tiles / f"{layer_name}.mbtiles").stat().st_size / 1e6
    print(f"Done — {size:.1f} MB")
else:
    print(result.stderr)

upload_to_mapbox(
    tiles / "hunting_stores.mbtiles",
    "hunting_stores",
    st_upload_write,
    st_upload_read
)

1063 POIs saved to C:\data\repos\onx_hunt\data\poi\hunting_stores.geojson
1063 → 514 POIs after clipping to NY/NJ/PA


C:\Users\culms\AppData\Local\Temp\ipykernel_17220\3855057227.py:57: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  poi_clipped = gpd.sjoin(poi, states[['geometry']], how='inner', predicate='within')


Done — 0.8 MB
Uploading hunting_stores.mbtiles → keystonecarto.hunting_stores
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4f9k41kcl1nqzlys27rut)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.hunting_stores


'keystonecarto.hunting_stores'

## Normalize the Regional Data

In [ ]:
# Combine NJ parks/forests, NY DEC lands, PA state game lands into public_lands.geojson
import geopandas as gpd
from shapely.geometry import MultiPolygon
from pathlib import Path

out_dir = Path("../data/public_lands")

# --- NJ Parks and Forests ---
nj = gpd.read_file(out_dir / "nj/parks_and_forests.gpkg")
nj = nj[['NAME_LABEL', 'MANAGED_BY', 'USE_LABEL', 'GISACRES', 'geometry']].rename(columns={
    'NAME_LABEL': 'name',
    'MANAGED_BY': 'managed_by',
    'USE_LABEL':  'land_type',
    'GISACRES':   'acres',
})
nj['state'] = 'NJ'

# --- NY DEC Lands ---
ny = gpd.read_file(out_dir / "ny/dec_lands.gpkg")
ny = ny[['FACILITY', 'CATEGORY', 'ACRES', 'geometry']].rename(columns={
    'FACILITY': 'name',
    'CATEGORY': 'land_type',
    'ACRES': 'acres',
})
ny['state']      = 'NY'
ny['managed_by'] = 'NY DEC'

# --- PA State Game Lands ---
pa = gpd.read_file(out_dir / "pa/PGC_StateGameland202301.shp").to_crs(epsg=4326)
pa['name'] = 'State Game Land ' + pa['SGL'].astype(str).str.zfill(3)
pa = pa[['name', 'ACRES', 'geometry']].rename(columns={'ACRES': 'acres'})
pa['state'] = 'PA'
pa['land_type'] = 'State Game Land'
pa['managed_by'] = 'PA Game Commission'

# --- Combine and normalize geometry to MultiPolygon ---
combined = gpd.GeoDataFrame(
    gpd.pd.concat([nj, ny, pa], ignore_index=True),
    crs='EPSG:4326'
)[['state', 'land_type', 'name', 'managed_by', 'acres', 'geometry']]

def to_multi(geom):
    if geom is None or geom.is_empty:
        return None
    return MultiPolygon([geom]) if geom.geom_type == 'Polygon' else geom

combined['geometry'] = combined['geometry'].apply(to_multi)
combined = combined[combined['geometry'].notna()].reset_index(drop=True)

out_path = out_dir / "public_lands.geojson"
combined.to_file(out_path, driver='GeoJSON')
print(f"Saved {len(combined)} features → {out_path}")
print(combined.groupby('state').size())

Saved 12268 features → ..\data\public_lands\public_lands.geojson
state
NJ    8726
NY    3233
PA     309
dtype: int64


In [18]:
run_tippecanoe([
    ("public_lands", "data/public_lands/public_lands.geojson", 5, 13),
])

upload_to_mapbox(
    tiles / "public_lands.mbtiles",
    "public_lands",
    st_upload_write,
    st_upload_read
)

Running Tippecanoe: public_lands...
  Done — 10.3 MB
Uploading public_lands.mbtiles → keystonecarto.public_lands
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4gwll1k2g1np5e0vovq88)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.public_lands


'keystonecarto.public_lands'

In [ ]:
# Merge the hunting units

import geopandas as gpd
from shapely.geometry import MultiPolygon
from pathlib import Path

# --- Config: (file, zone_type label, id column) ---
sources = [
    ("../data/hunting_units/nj/deer_management_zones.gpkg", "DMZ", "DMZ"),
    ("../data/hunting_units/ny/wildlife_management_units.gpkg", "UNIT", "UNIT"),
    ("../data/hunting_units/pa/PGC_BNDWildlifeManagementUnits2021.shp", "WMU_ID", "WMU_ID"),
]

# Generalize this function and ship up top
def to_multipolygon(geom):
    """Normalize Polygon → MultiPolygon so all features share one geometry type."""
    if geom is None:
        return None
    if geom.geom_type == "Polygon":
        return MultiPolygon([geom])
    return geom

layers = []
for path, zone_type, id_col in sources:
    gdf = gpd.read_file(path)

    # Reproject to WGS84 if needed
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    # Normalize geometry type to MultiPolygon
    gdf["geometry"] = gdf["geometry"].apply(to_multipolygon)

    # Add zone_type + zone_id, drop everything else
    gdf["zone_type"] = zone_type
    gdf["zone_id"]   = gdf[id_col].astype(str)
    gdf = gdf[["zone_type", "zone_id", "geometry"]]

    layers.append(gdf)
    print(f"{zone_type:6s}: {len(gdf)} features")

# --- Combine and save ---
combined = gpd.GeoDataFrame(
    gpd.pd.concat(layers, ignore_index=True),
    crs="EPSG:4326"
)

# out_path = Path("../data/hunting_units/hunting_units.gpkg")
out_path = Path("../data/hunting_units/hunting_units.geojson")
out_path.parent.mkdir(parents=True, exist_ok=True)
# combined.to_file(out_path, driver="GPKG")
combined.to_file(out_path, driver="GeoJSON")
print(f"\nSaved {len(combined)} total features → {out_path.resolve()}")

DMZ   : 90 features
UNIT  : 92 features
WMU_ID: 23 features

Saved 205 total features → C:\data\repos\onx_hunt\data\hunting_units\hunting_units.geojson


In [20]:
run_tippecanoe([
    ("hunting_units",     "/data/hunting_units/hunting_units.geojson",           0, 9),
])

upload_to_mapbox(
    tiles / "hunting_units.mbtiles",
    "hunting_units",
    st_upload_write,
    st_upload_read
)


Running Tippecanoe: hunting_units...
  Done — 0.7 MB
Uploading hunting_units.mbtiles → keystonecarto.hunting_units
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4i79e1kqq1pqz2dcqsj5u)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.hunting_units


'keystonecarto.hunting_units'

In [ ]:
# Create the MNO/AT&T tiles here since the rest of the code is here

run_tippecanoe([
    ("att_res6",     "scraped_data/mno/att/h3/att_res6.geojson",           0, 7),
    ("att_res7",     "scraped_data/mno/att/h3/att_res7.geojson",           7, 9),
    ("att_res8",     "scraped_data/mno/att/h3/att_res8.geojson",           9, 11),
])

layers_to_upload = [
    ("att_res6",     "scraped_data/mno/att/h3/att_res6.geojson",           0, 7),
    ("att_res7",     "scraped_data/mno/att/h3/att_res7.geojson",           7, 9),
    ("att_res8",     "scraped_data/mno/att/h3/att_res8.geojson",          9, 11),
]

for layer_name, _, _, _ in layers_to_upload:
    mbtiles_path = tiles / f"{layer_name}.mbtiles"
    if mbtiles_path.exists():
        upload_to_mapbox(mbtiles_path, layer_name, st_upload_write, st_upload_read)
    else:
        print(f"Skipping {layer_name} — .mbtiles not found, run Tippecanoe cell first")

Running Tippecanoe: att_res6...
  Done — 2.2 MB
Running Tippecanoe: att_res7...
  Done — 4.3 MB
Running Tippecanoe: att_res8...
  Done — 20.1 MB
Uploading att_res6.mbtiles → keystonecarto.att_res6
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4n0v30ajb1qs52zufqg0y)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.att_res6
Uploading att_res7.mbtiles → keystonecarto.att_res7
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4nl6o13fo1oo6pk8ix89c)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.att_res7
Uploading att_res8.mbtiles → keystonecarto.att_res8
  Uploading to S3 staging...
  S3 upload complete
  Processing started (upload id: cmls4odzz08rv1pphkkhccacr)
  Processing... (0%)
  Processing... (0%)
  Processing... (0%)
  Done — tileset ready: mapbox://keystonecarto.att_res8


## PA Harvest Lookup
Aids us in styling and creating additional layers within the map without recreating the tiles multiple times.

In [25]:
import geopandas as gpd
import json
from pathlib import Path

harvest = gpd.read_file("../data/misc/harvest/harvest_v1.gpkg")

print(f"Years: {sorted(harvest['LicenseYear'].dropna().unique().astype(int).tolist())}")

# Aggregate per year × WMU
agg = (
    harvest
    .groupby(["LicenseYear", "WMU_ID"])["Antlered_Deer_Harvested_per_Squ"]
    .mean()
    .round(3)
)

# Nested dict: { "2009": { "2B": 3.21, ... }, "2010": { ... }, ... }
by_year = {}
for (year, wmu_id), val in agg.items():
    yr = str(int(year))
    by_year.setdefault(yr, {})[str(wmu_id)] = val

years = sorted(by_year.keys())
wmus  = sorted({wmu for data in by_year.values() for wmu in data})
print(f"Years: {years[0]} – {years[-1]}  |  WMUs: {len(wmus)}")

# Write as a JS file — works with file:// unlike fetch()
out_dir = Path("../maps/data")
out_dir.mkdir(parents=True, exist_ok=True)
js_path = out_dir / "pa_wmu_harvest.js"
js_path.write_text(f"const paHarvestData = {json.dumps(by_year, indent=2)};")

print(f"Saved → {js_path.resolve()}")


Years: [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years: 2009 – 2024  |  WMUs: 23
Saved → C:\data\repos\onx_hunt\maps\data\pa_wmu_harvest.js
